YOu can easily create a data type that implements an interface and hence can be used as an argument to any function that expects an instance of a data type that implements that interface.

For instance, you can create a one-hot vector, i.e., a vector that has a single non-zero component with value 1, all other components have value 0.  An instance of such a data type only needs to store the length of the vector, and the index of the non-zero (hot) component.

In [1]:
mutable struct OneHotVector <: AbstractVector{Bool}
    const length::Int
    one_index::Int
end

Note: the `length` member of the structure can be declared `const` since you will not modify the length once the vector is created (this requires Julia 1.8+).

You can make this data structure behave like a regular Julia `Vector` by implementing a few methods for the `Base.getindex`, `Base.setindex` and `size` functions respectively.

In [2]:
function Base.getindex(v::OneHotVector, index::Int)
    if index < 0 || index > length(v)
        throw(BoundsError(v, index))
    elseif index == v.one_index
        return one(eltype(v))
    else
        return zero(eltype(v))
    end
end

function Base.setindex!(v::OneHotVector, value::Bool, index::Int)
    if index < 0 || index > length(v)
        throw(BoundsError(v, index))
    elseif value
        v.one_index = index
    end
    return value
end

function Base.size(v::OneHotVector)
    return (v.length, )
end

Note: it is good practice to throw an error when the index of a component is out of range.  Also, the `size` function is expected to return a tuple, so a 1-tuple for onedimensional vectors.  It is also good practice to make `Base.setindex` return the value since that allows chaining assingments, which otherwise would fail.

Now you can use the `OneHotVector` mostly as if it were a regular `Vector`.  First, you create an instance of a `OneHotVector` with length 5 and a non-zero component at index 2.

In [11]:
one_hot = OneHotVector(5, 2)

5-element OneHotVector:
 0
 1
 0
 0
 0

The "components" can be accessed using the syntax you are familiar with.  Under the hood, this is translated to a call of the `Base.getindex` function.

In [12]:
one_hot[3]

false

Similarly, a component can be set by assignment, i.e., implicitely calling the `Base.setindex` function.

In [13]:
one_hot[4] = true

true

In [14]:
one_hot

5-element OneHotVector:
 0
 0
 0
 1
 0

Note that you can actually set the non-zero component to false.  However, this doesn't modify the one-hot vector in any way.  You may want to make `Base.setindex` treat that as an error.

In [15]:
one_hot[4] = false

false

In [16]:
one_hot

5-element OneHotVector:
 0
 0
 0
 1
 0

As you can see, the `length` function also works, since under the hood, it calls the `size` method you implemented.

In [17]:
length(one_hot)

5

The interface you implemented is that of `AbstractVector`.  This should make `OneHotVector` behave like `Vector`, so you should be able to perform computation with a `OneHotVector`.

In [7]:
M = Matrix{Float64}([
        3.1 4.2 5.3
        6.4 7.5 8.6
        ])

2×3 Matrix{Float64}:
 3.1  4.2  5.3
 6.4  7.5  8.6

In [8]:
one_hot = OneHotVector(3, 2)

3-element OneHotVector:
 0
 1
 0

In [9]:
M*one_hot

2-element Vector{Float64}:
 4.2
 7.5

Indeed, when you compute the product of a matrix and a `OneHotVector`, you get the expected result since `true` is interpreted as ` in a numerical context, and `false` as 0.